In [1]:
from pyspark import SQLContext, SparkContext, SparkConf
from pyspark.sql.functions import col
from pyspark.conf import SparkConf
from pyspark.mllib.regression import LabeledPoint
import numpy as np
from pyspark.mllib.regression import LinearRegressionWithSGD
from pyspark.mllib.tree import DecisionTree
import matplotlib
sqlContext = SQLContext(sc)


Creating SparkContext as 'sc'


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
23,None,pyspark,idle,,,✔


Creating HiveContext as 'sqlContext'
SparkContext and HiveContext created. Executing user code ...


In [2]:
df = sqlContext.read.json("In/reviews.json")
df.printSchema()

root
 |-- helpfuless_count: long (nullable = true)
 |-- helpfuless_score: long (nullable = true)
 |-- price: string (nullable = true)
 |-- productId: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- score: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- text: string (nullable = true)
 |-- time: string (nullable = true)
 |-- title: string (nullable = true)
 |-- userId: string (nullable = true)

In [3]:
df1 = df.select('helpfuless_score', 'helpfuless_count','price','score')
df1.show(5)

+----------------+----------------+--------+-----+
|helpfuless_score|helpfuless_count|   price|score|
+----------------+----------------+--------+-----+
|               7|               7| unknown|  4.0|
|               0|               0|   17.99|  5.0|
|               0|               1|   17.99|  3.0|
|               7|               7| unknown|  4.0|
|               3|               4|   15.99|  5.0|
+----------------+----------------+--------+-----+
only showing top 5 rows

In [4]:
df1 = df1[df1.helpfuless_score > 0]
df1 = df1[df1.helpfuless_count > 0]
df1 = df1[df1.price > 0]
df1 = df1[df1.score > 0]
df1.show()

+----------------+----------------+------+-----+
|helpfuless_score|helpfuless_count| price|score|
+----------------+----------------+------+-----+
|               3|               4| 15.99|  5.0|
|               8|              10| 19.40|  5.0|
|               1|               1| 19.40|  5.0|
|               1|               1| 19.40|  5.0|
|               1|               1| 19.40|  5.0|
|               4|               4| 10.26|  5.0|
|               1|               1| 10.26|  5.0|
|               7|              11| 10.95|  1.0|
|               1|               2| 10.95|  4.0|
|               1|               2| 10.95|  1.0|
|               2|               4| 10.95|  5.0|
|               5|               9| 10.95|  5.0|
|               1|               3| 10.95|  5.0|
|               1|               4| 10.95|  5.0|
|               1|               4| 10.95|  5.0|
|               4|               6| 10.95|  5.0|
|               2|               3| 10.95|  5.0|
|               1|  

In [5]:
df1 = df1.select('score', 'price','helpfuless_count','helpfuless_score')
df1.show(5)

+-----+------+----------------+----------------+
|score| price|helpfuless_count|helpfuless_score|
+-----+------+----------------+----------------+
|  5.0| 15.99|               4|               3|
|  5.0| 19.40|              10|               8|
|  5.0| 19.40|               1|               1|
|  5.0| 19.40|               1|               1|
|  5.0| 19.40|               1|               1|
+-----+------+----------------+----------------+
only showing top 5 rows

In [6]:
def get_mapping(rdd, idx):
    return rdd.map(lambda fields: fields[idx]).distinct().zipWithIndex().collectAsMap()

In [7]:
print "Mapping of first categorical feature column: %s" % get_mapping(df1, 0)

Mapping of first categorical feature column: {u' 3.0': 3, u' 4.0': 4, u' 1.0': 2, u' 2.0': 0, u' 5.0': 1}

In [8]:
mappings = [get_mapping(df1, i) for i in range(0,1)]
cat_len = sum(map(len, mappings))
num_len = len(df1.first()[1:2])
total_len = num_len + cat_len

print "Feature vector length for categorical features: %d" % cat_len
print "Feature vector length for numerical features: %d" % num_len
print "Total feature vector length: %d" % total_len


Feature vector length for categorical features: 5
Feature vector length for numerical features: 1
Total feature vector length: 6

In [9]:
def extract_features(record):
    cat_vec = np.zeros(cat_len)
    i = 0
    step = 0
    for field in record[0:0]:
        m = mappings[i]
        idx = m[field]
        cat_vec[idx + step] = 1
        i = i + 1
        step = step + len(m)
    num_vec = np.array([float(field) for field in record[1:2]])
    return np.concatenate((cat_vec, num_vec))
def extract_label(record):
    return float(record[-1])

In [10]:
data = df1.map(lambda r: LabeledPoint(extract_label(r), extract_features(r)))

In [11]:
first = df1.first()
first_point = data.first()
print "Raw data: " + str(first[0:])
print "Label: " + str(first_point.label)
print "Linear Model feature vector:\n" + str(first_point.features)
print "Linear Model feature vector length: " + str(len(first_point.features))

Raw data: (u' 5.0', u' 15.99', 4, 3)
Label: 3.0
Linear Model feature vector:
[0.0,0.0,0.0,0.0,0.0,15.99]
Linear Model feature vector length: 6

In [1]:
def extract_features_dt(record):
    return np.array(map(float, record[0:2]))
data_dt = df1.map(lambda r: LabeledPoint(extract_label(r), extract_features_dt(r)))
first_point_dt = data_dt.first()
print "Decision Tree feature vector: " + str(first_point_dt.features)
print "Decision Tree feature vector length: " + str(len(first_point_dt.features))

Creating SparkContext as 'sc'


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
28,None,pyspark,idle,,,✔


Creating HiveContext as 'sqlContext'
SparkContext and HiveContext created. Executing user code ...


name 'df1' is not defined
Traceback (most recent call last):
NameError: name 'df1' is not defined



JAM edit below 

In [ ]:
dt_model = DecisionTree.trainRegressor(data_dt,{})
preds = dt_model.predict(data_dt.map(lambda p: p.features))
actual = data.map(lambda p: p.label)
true_vs_predicted_dt = actual.zip(preds)
print "Decision Tree predictions: " + str(true_vs_predicted_dt.take(5))
print "Decision Tree depth: " + str(dt_model.depth())
print "Decision Tree number of nodes: " + str(dt_model.numNodes())

In [ ]:
linear_model = LinearRegressionWithSGD.train(data, iterations=10, step=0.1, intercept=False)
true_vs_predicted = data.map(lambda p: (p.label, linear_model.predict(p.features)))
print "Linear Model predictions: " + str(true_vs_predicted.take(5))

In [ ]:
def squared_error(actual, pred):
    return (pred - actual)**2

In [ ]:
def abs_error(actual, pred):
    return np.abs(pred - actual)

In [ ]:
def squared_log_error(pred, actual):
    return (np.log(pred + 1) - np.log(actual + 1))**2

In [ ]:
mse = true_vs_predicted.map(lambda (t, p): squared_error(t, p)).mean()
mae = true_vs_predicted.map(lambda (t, p): abs_error(t, p)).mean()
rmsle = np.sqrt(true_vs_predicted.map(lambda (t, p): squared_log_error(t, p)).mean())
print "Linear Model - Mean Squared Error: %2.4f" % mse
print "Linear Model - Mean Absolute Error: %2.4f" % mae
print "Linear Model - Root Mean Squared Log Error: %2.4f" % rmsle

In [ ]:
mse_dt = true_vs_predicted_dt.map(lambda (t, p): squared_error(t, p)).mean()
mae_dt = true_vs_predicted_dt.map(lambda (t, p): abs_error(t, p)).mean()
rmsle_dt = np.sqrt(true_vs_predicted_dt.map(lambda (t, p): squared_log_error(t, p)).mean())
print "Decision Tree - Mean Squared Error: %2.4f" % mse_dt
print "Decision Tree - Mean Absolute Error: %2.4f" % mae_dt
print "Decision Tree - Root Mean Squared Log Error: %2.4f" % rmsle_dt